In [4]:
import os
import pickle
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import metrics

import sqlalchemy
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import *

import xgboost as xgb
from datetime import datetime, timedelta
import time  
import pytz    

import Preprocessing

import warnings
warnings.filterwarnings('ignore')

In [5]:
snowflake_creds = {
    'USER_NAME': 'omarhamzic',
    'PASSWORD': 'Werewolf123!@#'
}

In [8]:
# Creating the connection engine (way 1)
engine = create_engine(URL(
        account="cr21746.ap-south-1",
        user= snowflake_creds['USER_NAME'],
        password= snowflake_creds['PASSWORD'],
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="HEALTHSCHEMA"
    ))

In [9]:
def retraining_batch_query(a):
    query = f"""

        WITH TRAIN_BASE AS (

            SELECT CASE_ID,
                   COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
                   COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
                   COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
                   COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
                   COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
                   COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
                   COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
                   COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
                   COALESCE(BED_GRADE,0) AS BED_GRADE,
                   PATIENTID,
                   COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
                   COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
                   COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
                   COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
                   COALESCE(AGE,'None') AS AGE,
                   COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
                   ADMISSION_DATE,
                   DISCHARGE_DATE

            FROM HEALTHDB.HEALTHSCHEMA.HEALTH_DATA
            WHERE ADMISSION_DATE >= '2022-11-01' --- to reduce the load

        ),

        TRAIN_BASE_WITH_FEATURES AS (

            SELECT *,
                    MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
                    DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,    
                    CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
                    CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
                    CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
                    DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
            FROM TRAIN_BASE 

        ),    

        NEW_DATA_WITH_FEATURES AS (

             SELECT CASE_ID,
                       COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
                       COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
                       COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
                       COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
                       COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_X,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
                       COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
                       COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
                       COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
                       COALESCE(BED_GRADE,0) AS BED_GRADE,
                       PATIENTID,
                       COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
                       COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
                       COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
                       COALESCE(VISITORS_WITH_PATIENT_X,0) AS VISITORS_WITH_PATIENT,
                       COALESCE(AGE,'None') AS AGE,
                       COALESCE(ADMISSION_DEPOSIT_X,0) AS ADMISSION_DEPOSIT,
                       ADMISSION_DATE,
                       DISCHARGE_DATE,
                       ADMISSION_MONTH,
                       ADMISSION_DAY,
                       ADMISSION_ILLNESS,
                       ILLNESS_BEDGRADE,
                       DEPARTMENT_ILLNESS,
                       LOS_X AS LOS 
                    FROM HEALTHDB.HEALTHSCHEMA.TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI
                    WHERE ADMISSION_DATE >= CURRENT_DATE-144+{a*7} AND ADMISSION_DATE < CURRENT_DATE-144+{(a+1)*7}    

        )


        SELECT * FROM TRAIN_BASE_WITH_FEATURES
        UNION ALL
        SELECT * FROM NEW_DATA_WITH_FEATURES;

        """
    return query

In [ ]:
with engine.connect() as conn:
    data = pd.DataFrame(pd.read_sql(retraining_batch_query(0),conn))
    data.columns = [col.upper() for col in data.columns.tolist()]

In [ ]:
print(data.shape)
data.head()

In [ ]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test

In [ ]:
def feature_selection(df):
    # Creating X and Y
    x_train = df.drop('LOS',axis=1)
    y_train = df[['LOS']]
    
    # Decision Tree
    from sklearn.tree import DecisionTreeRegressor

    dtree = DecisionTreeRegressor()
    dtree.fit(x_train,y_train)
    # Feature Importance
    feat_imp = (pd.DataFrame(zip(x_train.columns,dtree.feature_importances_),columns=['feature','imp'])
                .sort_values(by='imp',ascending=False))
    final_features_dtree = feat_imp[feat_imp['imp']>=0.01]['feature'].values.tolist()
    
    # XGBoost
    import xgboost as xgb

    xgb_ = xgb.XGBRegressor()
    xgb_.fit(x_train,y_train)
    # Feature Importance
    feat_imp = (pd.DataFrame(zip(x_train.columns,xgb_.feature_importances_),columns=['feature','imp'])
                .sort_values(by='imp',ascending=False))
    final_features_xgb = feat_imp[feat_imp['imp']>=0.01]['feature'].values.tolist()
    
    model_features =  list(set(final_features_dtree).union(set(final_features_xgb)))
    print("Final Features from both Dtree & XGB: "+str(len(model_features)))
    
    import pickle

    final_feats_list = model_features+['LOS']

    with open('./Retraining Artifacts/MODEL_FEATS.pkl','wb') as F:
        pickle.dump(final_feats_list,F)
    
    return final_feats_list
    

In [ ]:
#model_feats.remove('LOS')

In [3]:
def load_scoring_data(conn):
    # Loading the scoring data
    data = pd.DataFrame(pd.read_sql(retraining_batch_query(0), conn))
    data.columns = [col.upper() for col in data.columns.tolist()]
    return data

def split_data_by_date(data):
    # Splitting the data into Train and Test set
    tz_NY = pytz.timezone('America/New_York')
    max_date = data.ADMISSION_DATE.max()
    min_date = max_date - timedelta(days=7)
    data_train = data[data['ADMISSION_DATE'] <= min_date]
    data_test = data[(data['ADMISSION_DATE'] >= min_date) & (data['ADMISSION_DATE'] <= max_date)]
    return data_train, data_test

def preprocess_data(data):
    # Applying preprocessing steps
    df_processed = Preprocessing.preprocess_data(data)
    return df_processed

def train_model(df_train, model_feats):
    # Performing feature selection
    model = xgb.XGBRegressor()
    model.fit(df_train[model_feats], df_train['LOS'])
    return model

def evaluate_model(model, df_test, model_feats):
    # Evaluating the model
    df_test_final = check_n_create_model_features(df_test, model_feats)
    if 'LOS' in df_test_final.columns.tolist():
        df_test_final = df_test_final.drop('LOS', axis=1)
    preds = np.ceil(model.predict(df_test_final))
    rmse = np.sqrt(metrics.mean_squared_error(df_test['LOS'], preds))
    mae = np.sqrt(metrics.mean_absolute_error(df_test['LOS'], preds))
    return rmse, mae

def save_trained_model(model, model_metrics):
    # Saving the trained model and its performance metrics
    model.get_booster().save_model('./Retraining Artifacts/MODEL_XGB.model')
    with open('./Retraining Artifacts/MODEL_XGB_PERFM_METRICS.pkl', 'wb') as F:
        pickle.dump(model_metrics, F)

def load_old_model_and_predict(df_test, model_feats_old):
    # Loading the old model and making predictions
    model = xgb.XGBRegressor()
    model.load_model('MODEL_XGB.model')
    df_test_final = check_n_create_model_features(df_test, model_feats_old)
    if 'LOS' in df_test_final.columns.tolist():
        df_test_final = df_test_final.drop('LOS', axis=1)
    preds = np.ceil(model.predict(df_test_final))
    rmse = np.sqrt(metrics.mean_squared_error(df_test['LOS'], preds))
    mae = np.sqrt(metrics.mean_absolute_error(df_test['LOS'], preds))
    return rmse, mae

if __name__ == "__main__":
    # Establish a database connection (engine)
    with engine.connect() as conn:
        # Load scoring data
        data = load_scoring_data(conn)

        # Split data into train and test sets
        data_train, data_test = split_data_by_date(data)

        # Preprocess the data
        df_train_processed = preprocess_data(data_train)
        df_test_processed = preprocess_data(data_test)

        # Train the model
        model_feats = feature_selection(df_train_processed)
        model_feats.remove('LOS')
        model = train_model(df_train_processed, model_feats)

        # Evaluate the new model
        rmse_new, mae_new = evaluate_model(model, df_test_processed, model_feats)
        print("\nTest Performance (new model)")
        print("RMSE:", rmse_new)
        print("MAE:", mae_new)

        # Save the trained model and its performance metrics
        model_xgb_metrics_new = {'RMSE': rmse_new, 'MAE': mae_new}
        save_trained_model(model, model_xgb_metrics_new)
        
        # Load the old model and evaluate
        with open('MODEL_FEATS.pkl','rb') as F:
            model_feats_old = pickle.load(F)
        rmse_old, mae_old = load_old_model_and_predict(df_test_processed, model_feats_old)
        print("\nTest Performance (old model)")
        print("RMSE:", rmse_old)
        print("MAE:", mae_old)


NameError: name 'engine' is not defined

In [ ]:
model.n_features_in_

In [ ]:
len(model_feats_old)

# Retraining Final Functions/Scripts:

In [ ]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb

def feature_selection(df):
    # Creating X and Y
    x_train = df.drop('LOS', axis=1)
    y_train = df['LOS']

    # Decision Tree
    dtree = DecisionTreeRegressor()
    dtree.fit(x_train, y_train)
    
    # Feature Importance for Decision Tree
    feat_imp_dtree = pd.DataFrame(zip(x_train.columns, dtree.feature_importances_), columns=['feature', 'imp'])
    final_features_dtree = feat_imp_dtree[feat_imp_dtree['imp'] >= 0.01]['feature'].tolist()

    # XGBoost
    xgb_ = xgb.XGBRegressor()
    xgb_.fit(x_train, y_train)

    # Feature Importance for XGBoost
    feat_imp_xgb = pd.DataFrame(zip(x_train.columns, xgb_.feature_importances_), columns=['feature', 'imp'])
    final_features_xgb = feat_imp_xgb[feat_imp_xgb['imp'] >= 0.01]['feature'].tolist()

    # Combine features from both models
    model_features = list(set(final_features_dtree).union(set(final_features_xgb)))
    print("Final Features from both Decision Tree and XGBoost: " + str(len(model_features)))

    # Save the final feature list including the target variable 'LOS'
    final_feats_list = model_features + ['LOS']

    with open('./Retraining Artifacts/MODEL_FEATS.pkl', 'wb') as F:
        pickle.dump(final_feats_list, F)

    return final_feats_list

In [ ]:
def retraining_batch_query(max_date):
    query = f"""

        WITH TRAIN_BASE AS (

            SELECT CASE_ID,
                   COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
                   COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
                   COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
                   COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
                   COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
                   COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
                   COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
                   COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
                   COALESCE(BED_GRADE,0) AS BED_GRADE,
                   PATIENTID,
                   COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
                   COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
                   COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
                   COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
                   COALESCE(AGE,'None') AS AGE,
                   COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
                   ADMISSION_DATE,
                   DISCHARGE_DATE

            FROM HEALTHDB.HEALTHSCHEMA.HEALTH_DATA
            WHERE ADMISSION_DATE >= '2022-11-01' --- To reduce the load

        ),

        TRAIN_BASE_WITH_FEATURES AS (

            SELECT *,
                    MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
                    DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,    
                    CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
                    CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
                    CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
                    DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
            FROM TRAIN_BASE 

        ),    

        NEW_DATA_WITH_FEATURES AS (

             SELECT CASE_ID,
                       COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
                       COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
                       COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
                       COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
                       COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_X,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
                       COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
                       COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
                       COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
                       COALESCE(BED_GRADE,0) AS BED_GRADE,
                       PATIENTID,
                       COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
                       COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
                       COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
                       COALESCE(VISITORS_WITH_PATIENT_X,0) AS VISITORS_WITH_PATIENT,
                       COALESCE(AGE,'None') AS AGE,
                       COALESCE(ADMISSION_DEPOSIT_X,0) AS ADMISSION_DEPOSIT,
                       ADMISSION_DATE,
                       DISCHARGE_DATE,
                       ADMISSION_MONTH,
                       ADMISSION_DAY,
                       ADMISSION_ILLNESS,
                       ILLNESS_BEDGRADE,
                       DEPARTMENT_ILLNESS,
                       LOS_X AS LOS 
                    FROM HEALTHDB.HEALTHSCHEMA.TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI
                    WHERE ADMISSION_DATE < '{max_date}'

        )


        SELECT * FROM TRAIN_BASE_WITH_FEATURES
        UNION ALL
        SELECT * FROM NEW_DATA_WITH_FEATURES;

        """
    return query

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn import metrics
import xgboost as xgb
import pytz
from datetime import datetime, timedelta

def retrain_and_evaluate_model(cut_off_date):
    with engine.connect() as conn:
        # Loading the scoring data
        data = pd.DataFrame(pd.read_sql(retraining_batch_query(cut_off_date), conn))
        data.columns = [col.upper() for col in data.columns.tolist()]
    
    # Splitting the data into Train and Test set
    tz_NY = pytz.timezone('America/New_York')
    max_date = data.ADMISSION_DATE.max()
    min_date = max_date - timedelta(days=7)
    data_train = data[data['ADMISSION_DATE'] <= min_date]
    data_test = data[(data['ADMISSION_DATE'] >= min_date) & (data['ADMISSION_DATE'] <= max_date)]

    # Preprocess the data
    df_train_processed = Preprocessing.preprocess_data(data_train)
    df_test_processed = Preprocessing.preprocess_data(data_test)

    # Performing feature selection
    df_final = df_train_processed.copy()
    model_feats = feature_selection(df_final)
    model_feats.remove('LOS')

    # Train the new model
    xgb_ = xgb.XGBRegressor()
    xgb_.fit(df_final[model_feats], df_final['LOS'])

    # Evaluate the new model
    df_test_final = check_n_create_model_features(df_test_processed, model_feats)
    if 'LOS' in df_test_final.columns.tolist():
        df_test_final = df_test_final.drop('LOS', axis=1)
    preds_new = np.ceil(xgb_.predict(df_test_final))
    rmse_new = np.sqrt(metrics.mean_squared_error(df_test_processed['LOS'], preds_new))
    mae_new = np.sqrt(metrics.mean_absolute_error(df_test_processed['LOS'], preds_new))

    # Save the trained new model and its performance metrics
    booster = xgb_.get_booster()
    booster.save_model('./Retraining Artifacts/MODEL_XGB.model')
    model_xgb_metrics_new = {'RMSE': rmse_new, 'MAE': mae_new}
    with open('./Retraining Artifacts/MODEL_XGB_PERFM_METRICS.pkl', 'wb') as F:
        pickle.dump(model_xgb_metrics_new, F)

    # Load the old model and evaluate
    model = xgb.XGBRegressor()
    model.load_model('MODEL_XGB.model')
    with open('MODEL_FEATS.pkl', 'rb') as F:
        model_feats_old = pickle.load(F)
    df_test_final = check_n_create_model_features(df_test_processed, model_feats_old)
    if 'LOS' in df_test_final.columns.tolist():
        df_test_final = df_test_final.drop('LOS', axis=1)
    preds_old = np.ceil(model.predict(df_test_final))
    rmse_old = np.sqrt(metrics.mean_squared_error(df_test_processed['LOS'], preds_old))
    mae_old = np.sqrt(metrics.mean_absolute_error(df_test_processed['LOS'], preds_old))
    
    # Create dictionaries for performance metrics
    model_xgb_metrics_old = {'RMSE': rmse_old, 'MAE': mae_old}

    return model_xgb_metrics_new, model_xgb_metrics_old

In [ ]:
new_dict, old_dict = retrain_and_evaluate_model('2022-12-30')

In [ ]:
def finalize_model(new_perform_dict, old_perform_dict):
    count = 0
    for metric in new_perform_dict.keys():
        if new_perform_dict[metric] < old_perform_dict[metric]:
            count += 1
    
    if count > 0:
        return 'New Model'
    else:
        return 'Old Model'

In [ ]:
finalize_model(new_perform_dict=new_dict, old_perform_dict=old_dict)

In [ ]:
import shutil

def deploy_model(selector='Old Model'):
    if selector != 'Old Model':
        # Define paths
        archive_folder = './Archive/'
        retraining_folder = './Retraining Artifacts/'

        # Define files to be archived
        files_to_archive = [
            'MODEL_FEATS.pkl',
            'MODEL_XGB.model',
            'MODEL_XGB_PERFM_METRICS.pkl'
        ]

        # Archive old model files
        for filename in files_to_archive:
            old_filepath = filename
            archive_filepath = archive_folder + filename
            shutil.move(old_filepath, archive_filepath)

        # Replace with new model artifacts
        new_files_to_replace = [
            'MODEL_FEATS.pkl',
            'MODEL_XGB.model',
            'MODEL_XGB_PERFM_METRICS.pkl'
        ]

        # Replace old model files with new model files
        for filename in new_files_to_replace:
            new_filepath = retraining_folder + filename
            old_filepath = filename
            shutil.move(new_filepath, old_filepath)

    return 'Deployment Successful'


In [ ]:
deploy_model('Old Model')